# Assignment 2, 8-Puzzle Programming - A Star

### Your homework must be implemented in this Notebook file. 
### You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
### You need to implement the two searching functions and the print result functions.
### For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.


In [193]:
t1 = 0
t2 = 0

In [194]:
#implementation of function "A Star Misplaced"
import heapq
import time

class Misplaced:
    def __init__(self, board, g=0, h=0):
        self.board = board
        self.gn = g  
        self.hn = h 
        self.fn = g + h  
        self.parent = None 
        
    def valid_moves(self):
        moves = []
        up = -3
        down = 3
        left = -1
        right = 1
        blank = self.board.index(0)
        if blank not in (0, 1, 2):
            moves.append(up)
        if blank not in (6, 7, 8):
            moves.append(down)
        if blank not in (0, 3, 6):
            moves.append(left)
        if blank not in (2, 5, 8):
            moves.append(right)
        return moves
        
    # We do the less than operation here.   
    def __lt__(self, compare):
        return self.fn < compare.fn

    def final_state(self):
        return self.board == (0, 1, 2, 3, 4, 5, 6, 7, 8)
    
    def shuffle(self, direction):
        new_board = list(self.board)
        i = new_board.index(0)
        j = i + direction
        new_board[i], new_board[j] = new_board[j], new_board[i]
        return Misplaced(tuple(new_board), self.gn + 1, self.heuristic(tuple(new_board)))

    def heuristic(self, board):
        return sum(1 if board[i] != goal[i] else 0 for i in range(9))

    def solve(self):
        global t1
        start_time = time.time()
        successor_list = [self]
        compare_list = set()
        steps = 0
        while successor_list:
            steps += 1
            current_node = heapq.heappop(successor_list)
            if current_node.final_state():                             
                path = [current_node]
                t1 += time.time() - start_time
                while current_node.parent:
                    path.append(current_node.parent)
                    current_node = current_node.parent
                return steps, path[::-1]            
            compare_list.add(current_node.board)
            for move in current_node.valid_moves():
                next_board = current_node.shuffle(move)
                if next_board.board not in compare_list:
                    next_board.parent = current_node
                    heapq.heappush(successor_list, next_board)               
        # print("Time: ", t1)
        return steps, None

    def print_board(self):
        board = list(self.board)
        for i in range(0, 9, 3):
            print(board[i:i+3])
        print("")

goal = (0, 1, 2, 3, 4, 5, 6, 7, 8)

In [195]:
#implementation of function "A Star Manhattan Distance"
import heapq
import time

class Manhattan:
    def __init__(self, board, parent=None, g=0, h=0):
        self.board = board
        self.parent = parent
        self.gn = g
        self.hn = h
        
    def __lt__(self, other):
        return self.gn + self.hn < other.gn + other.hn
    
    def goal_state(self):
        return self.board == (0, 1, 2, 3, 4, 5, 6, 7, 8)
    
    def valid_moves(self):
        moves = []
        up = -3
        down = 3
        left = -1
        right = 1
        zero_index = self.board.index(0)
        if zero_index >= 3:
            moves.append(up)
        if zero_index <= 5:
            moves.append(down)
        if zero_index % 3 != 0:
            moves.append(left)
        if zero_index % 3 != 2:
            moves.append(right)
        return moves
    
    def shuffle(self, move):
        zero_index = self.board.index(0)
        new_board = list(self.board)
        new_board[zero_index], new_board[zero_index + move] = new_board[zero_index + move], new_board[zero_index]
        return Manhattan(tuple(new_board), parent=self, g=self.gn + 1, h=self.manhattan_distance(new_board))
    
    def manhattan_distance(self, board):
        distance = 0
        for i in range(9):
            if board[i] == 0:
                continue
            x, y = divmod(i, 3)
            x_goal, y_goal = divmod(board[i], 3)
            distance += abs(x - x_goal) + abs(y - y_goal)
        return distance
    
    def print_board(self):
        board = list(self.board)
        for i in range(0, 9, 3):
            print(board[i:i+3])
        print("")
    
    def solve(self):
        global t2
        start_time = time.time()
        successor_list = [self]
        compare_list = set()
        steps2 = 0
        path = []
        while successor_list:
            current_node = heapq.heappop(successor_list)
            steps2 += 1
            if current_node.goal_state():
                t2 += time.time() - start_time                
                path.append(current_node)
                while current_node.parent:
                    current_node = current_node.parent
                    path.append(current_node)
                return steps, path[::-1]
            compare_list.add(current_node.board)
            for move in current_node.valid_moves():
                next_board = current_node.shuffle(move)
                if next_board.board not in compare_list:
                    next_board.parent = current_node
                    successor_list.append(next_board)
                    heapq.heapify(successor_list)
        return steps2, None


In [196]:
#implementation of function "print_result(result)"

In [197]:
def print_result():
    with open('Input8PuzzleCases.txt') as file:
        listRead = file.readlines()
        for i in range(100):            
        #string to list        
            list_input = listRead[i].split(',')
            list_input = [int(x) for x in list_input]
#             print("Starting State: ",list_input)
#             print("Input: ",tuple(list_input))    
            puzzle1 = Misplaced(tuple(list_input))
            steps, solution = puzzle1.solve()
            for i, state in enumerate(solution):
                    print("Step", i + 1, ":")
                    state.print_board()       
    file.close()
    with open('Input8PuzzleCases.txt') as file:
        listRead = file.readlines()
        for i in range(100):            
        #string to list        
            list_input = listRead[i].split(',')
            list_input = [int(x) for x in list_input]
#             print("Starting State: ",list_input)
#             print("Input: ",tuple(list_input))    
            puzzle2 = Manhattan(tuple(list_input))
            Mansteps, solution = puzzle2.solve()    
            if solution:
                for i, state in enumerate(solution):
                    print("Step", i + 1, ":")
                    state.print_board()            
            else:
                print("No solution found")
    print("\t\t" + "Average_steps" + "\t" + "Average_time")    
    print("MisPlaced: " + "\t"+ str(round(steps/100,0))+ "\t\t"+ str(round(t1/10,2))+ " seconds")
    print("Manhattan: " + "\t"+ str(round(Mansteps/100,0))+ "\t\t"+ str(round(t2/10,2))+ " seconds")

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
## ===============================

In [198]:
#you need to implement print_result function to print out the result according to the required format
print_result()

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]
[3, 1, 6]

Step 5 :
[8, 0, 7]
[4, 2, 5]
[3, 1, 6]

Step 6 :
[8, 2, 7]
[4, 0, 5]
[3, 1, 6]

Step 7 :
[8, 2, 7]
[4, 1, 5]
[3, 0, 6]

Step 8 :
[8, 2, 7]
[4, 1, 5]
[3, 6, 0]

Step 9 :
[8, 2, 7]
[4, 1, 0]
[3, 6, 5]

Step 10 :
[8, 2, 0]
[4, 1, 7]
[3, 6, 5]

Step 11 :
[8, 0, 2]
[4, 1, 7]
[3, 6, 5]

Step 12 :
[0, 8, 2]
[4, 1, 7]
[3, 6, 5]

Step 13 :
[4, 8, 2]
[0, 1, 7]
[3, 6, 5]

Step 14 :
[4, 8, 2]
[1, 0, 7]
[3, 6, 5]

Step 15 :
[4, 0, 2]
[1, 8, 7]
[3, 6, 5]

Step 16 :
[0, 4, 2]
[1, 8, 7]
[3, 6, 5]

Step 17 :
[1, 4, 2]
[0, 8, 7]
[3, 6, 5]

Step 18 :
[1, 4, 2]
[3, 8, 7]
[0, 6, 5]

Step 19 :
[1, 4, 2]
[3, 8, 7]
[6, 0, 5]

Step 20 :
[1, 4, 2]
[3, 0, 7]
[6, 8, 5]

Step 21 :
[1, 4, 2]
[3, 7, 0]
[6, 8, 5]

Step 22 :
[1, 4, 2]
[3, 7, 5]
[6, 8, 0]

Step 23 :
[1, 4, 2]
[3, 7, 5]
[6, 0, 8]

Step 24 :
[1, 4, 2]
[3, 0, 5]
[6, 7, 8]

Step 25 :
[1, 0, 2]
[3, 4

[0, 6, 1]
[3, 4, 7]
[8, 5, 2]

Step 2 :
[3, 6, 1]
[0, 4, 7]
[8, 5, 2]

Step 3 :
[3, 6, 1]
[4, 0, 7]
[8, 5, 2]

Step 4 :
[3, 6, 1]
[4, 7, 0]
[8, 5, 2]

Step 5 :
[3, 6, 1]
[4, 7, 2]
[8, 5, 0]

Step 6 :
[3, 6, 1]
[4, 7, 2]
[8, 0, 5]

Step 7 :
[3, 6, 1]
[4, 7, 2]
[0, 8, 5]

Step 8 :
[3, 6, 1]
[0, 7, 2]
[4, 8, 5]

Step 9 :
[3, 6, 1]
[7, 0, 2]
[4, 8, 5]

Step 10 :
[3, 0, 1]
[7, 6, 2]
[4, 8, 5]

Step 11 :
[3, 1, 0]
[7, 6, 2]
[4, 8, 5]

Step 12 :
[3, 1, 2]
[7, 6, 0]
[4, 8, 5]

Step 13 :
[3, 1, 2]
[7, 6, 5]
[4, 8, 0]

Step 14 :
[3, 1, 2]
[7, 6, 5]
[4, 0, 8]

Step 15 :
[3, 1, 2]
[7, 6, 5]
[0, 4, 8]

Step 16 :
[3, 1, 2]
[0, 6, 5]
[7, 4, 8]

Step 17 :
[3, 1, 2]
[6, 0, 5]
[7, 4, 8]

Step 18 :
[3, 1, 2]
[6, 4, 5]
[7, 0, 8]

Step 19 :
[3, 1, 2]
[6, 4, 5]
[0, 7, 8]

Step 20 :
[3, 1, 2]
[0, 4, 5]
[6, 7, 8]

Step 21 :
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]
[3, 1, 6

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]
[3, 1, 6]

Step 5 :
[8, 0, 7]
[4, 2, 5]
[3, 1, 6]

Step 6 :
[8, 2, 7]
[4, 0, 5]
[3, 1, 6]

Step 7 :
[8, 2, 7]
[4, 1, 5]
[3, 0, 6]

Step 8 :
[8, 2, 7]
[4, 1, 5]
[3, 6, 0]

Step 9 :
[8, 2, 7]
[4, 1, 0]
[3, 6, 5]

Step 10 :
[8, 2, 0]
[4, 1, 7]
[3, 6, 5]

Step 11 :
[8, 0, 2]
[4, 1, 7]
[3, 6, 5]

Step 12 :
[0, 8, 2]
[4, 1, 7]
[3, 6, 5]

Step 13 :
[4, 8, 2]
[0, 1, 7]
[3, 6, 5]

Step 14 :
[4, 8, 2]
[1, 0, 7]
[3, 6, 5]

Step 15 :
[4, 0, 2]
[1, 8, 7]
[3, 6, 5]

Step 16 :
[0, 4, 2]
[1, 8, 7]
[3, 6, 5]

Step 17 :
[1, 4, 2]
[0, 8, 7]
[3, 6, 5]

Step 18 :
[1, 4, 2]
[3, 8, 7]
[0, 6, 5]

Step 19 :
[1, 4, 2]
[3, 8, 7]
[6, 0, 5]

Step 20 :
[1, 4, 2]
[3, 0, 7]
[6, 8, 5]

Step 21 :
[1, 4, 2]
[3, 7, 0]
[6, 8, 5]

Step 22 :
[1, 4, 2]
[3, 7, 5]
[6, 8, 0]

Step 23 :
[1, 4, 2]
[3, 7, 5]
[6, 0, 8]

Step 24 :
[1, 4, 2]
[3, 0, 5]
[6, 7, 8]

Step 25 :
[1, 0, 2]
[3, 4

Step 1 :
[0, 6, 1]
[3, 4, 7]
[8, 5, 2]

Step 2 :
[3, 6, 1]
[0, 4, 7]
[8, 5, 2]

Step 3 :
[3, 6, 1]
[4, 0, 7]
[8, 5, 2]

Step 4 :
[3, 6, 1]
[4, 7, 0]
[8, 5, 2]

Step 5 :
[3, 6, 1]
[4, 7, 2]
[8, 5, 0]

Step 6 :
[3, 6, 1]
[4, 7, 2]
[8, 0, 5]

Step 7 :
[3, 6, 1]
[4, 7, 2]
[0, 8, 5]

Step 8 :
[3, 6, 1]
[0, 7, 2]
[4, 8, 5]

Step 9 :
[3, 6, 1]
[7, 0, 2]
[4, 8, 5]

Step 10 :
[3, 0, 1]
[7, 6, 2]
[4, 8, 5]

Step 11 :
[3, 1, 0]
[7, 6, 2]
[4, 8, 5]

Step 12 :
[3, 1, 2]
[7, 6, 0]
[4, 8, 5]

Step 13 :
[3, 1, 2]
[7, 6, 5]
[4, 8, 0]

Step 14 :
[3, 1, 2]
[7, 6, 5]
[4, 0, 8]

Step 15 :
[3, 1, 2]
[7, 6, 5]
[0, 4, 8]

Step 16 :
[3, 1, 2]
[0, 6, 5]
[7, 4, 8]

Step 17 :
[3, 1, 2]
[6, 0, 5]
[7, 4, 8]

Step 18 :
[3, 1, 2]
[6, 4, 5]
[7, 0, 8]

Step 19 :
[3, 1, 2]
[6, 4, 5]
[0, 7, 8]

Step 20 :
[3, 1, 2]
[0, 4, 5]
[6, 7, 8]

Step 21 :
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]

Step 1 :
[0, 6, 1]
[3, 4, 7]
[8, 5, 2]

Step 2 :
[3, 6, 1]
[0, 4, 7]
[8, 5, 2]

Step 3 :
[3, 6, 1]
[4, 0, 7]
[8, 5, 2]

Step 4 :
[3, 6, 1]
[4, 7, 0]
[8, 5, 2]

Step 5 :
[3, 6, 1]
[4, 7, 2]
[8, 5, 0]

Step 6 :
[3, 6, 1]
[4, 7, 2]
[8, 0, 5]

Step 7 :
[3, 6, 1]
[4, 7, 2]
[0, 8, 5]

Step 8 :
[3, 6, 1]
[0, 7, 2]
[4, 8, 5]

Step 9 :
[3, 6, 1]
[7, 0, 2]
[4, 8, 5]

Step 10 :
[3, 0, 1]
[7, 6, 2]
[4, 8, 5]

Step 11 :
[3, 1, 0]
[7, 6, 2]
[4, 8, 5]

Step 12 :
[3, 1, 2]
[7, 6, 0]
[4, 8, 5]

Step 13 :
[3, 1, 2]
[7, 6, 5]
[4, 8, 0]

Step 14 :
[3, 1, 2]
[7, 6, 5]
[4, 0, 8]

Step 15 :
[3, 1, 2]
[7, 6, 5]
[0, 4, 8]

Step 16 :
[3, 1, 2]
[0, 6, 5]
[7, 4, 8]

Step 17 :
[3, 1, 2]
[6, 0, 5]
[7, 4, 8]

Step 18 :
[3, 1, 2]
[6, 4, 5]
[7, 0, 8]

Step 19 :
[3, 1, 2]
[6, 4, 5]
[0, 7, 8]

Step 20 :
[3, 1, 2]
[0, 4, 5]
[6, 7, 8]

Step 21 :
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]

...

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]
[3, 1, 6]

Step 5 :
[8, 0, 7]
[4, 2, 5]
[3, 1, 6]

Step 6 :
[8, 2, 7]
[4, 0, 5]
[3, 1, 6]

Step 7 :
[8, 2, 7]
[4, 1, 5]
[3, 0, 6]

Step 8 :
[8, 2, 7]
[4, 1, 5]
[3, 6, 0]

Step 9 :
[8, 2, 7]
[4, 1, 0]
[3, 6, 5]

Step 10 :
[8, 2, 0]
[4, 1, 7]
[3, 6, 5]

Step 11 :
[8, 0, 2]
[4, 1, 7]
[3, 6, 5]

Step 12 :
[8, 1, 2]
[4, 0, 7]
[3, 6, 5]

Step 13 :
[8, 1, 2]
[0, 4, 7]
[3, 6, 5]

Step 14 :
[0, 1, 2]
[8, 4, 7]
[3, 6, 5]

Step 15 :
[1, 0, 2]
[8, 4, 7]
[3, 6, 5]

Step 16 :
[1, 4, 2]
[8, 0, 7]
[3, 6, 5]

Step 17 :
[1, 4, 2]
[0, 8, 7]
[3, 6, 5]

Step 18 :
[1, 4, 2]
[3, 8, 7]
[0, 6, 5]

Step 19 :
[1, 4, 2]
[3, 8, 7]
[6, 0, 5]

Step 20 :
[1, 4, 2]
[3, 0, 7]
[6, 8, 5]

Step 21 :
[1, 4, 2]
[3, 7, 0]
[6, 8, 5]

Step 22 :
[1, 4, 2]
[3, 7, 5]
[6, 8, 0]

Step 23 :
[1, 4, 2]
[3, 7, 5]
[6, 0, 8]

Step 24 :
[1, 4, 2]
[3, 0, 5]
[6, 7, 8]

Step 25 :
[1, 0, 2]
[3, 4

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]
[3, 1, 6]

Step 5 :
[8, 0, 7]
[4, 2, 5]
[3, 1, 6]

Step 6 :
[8, 2, 7]
[4, 0, 5]
[3, 1, 6]

Step 7 :
[8, 2, 7]
[4, 1, 5]
[3, 0, 6]

Step 8 :
[8, 2, 7]
[4, 1, 5]
[3, 6, 0]

Step 9 :
[8, 2, 7]
[4, 1, 0]
[3, 6, 5]

Step 10 :
[8, 2, 0]
[4, 1, 7]
[3, 6, 5]

Step 11 :
[8, 0, 2]
[4, 1, 7]
[3, 6, 5]

Step 12 :
[8, 1, 2]
[4, 0, 7]
[3, 6, 5]

Step 13 :
[8, 1, 2]
[0, 4, 7]
[3, 6, 5]

Step 14 :
[0, 1, 2]
[8, 4, 7]
[3, 6, 5]

Step 15 :
[1, 0, 2]
[8, 4, 7]
[3, 6, 5]

Step 16 :
[1, 4, 2]
[8, 0, 7]
[3, 6, 5]

Step 17 :
[1, 4, 2]
[0, 8, 7]
[3, 6, 5]

Step 18 :
[1, 4, 2]
[3, 8, 7]
[0, 6, 5]

Step 19 :
[1, 4, 2]
[3, 8, 7]
[6, 0, 5]

Step 20 :
[1, 4, 2]
[3, 0, 7]
[6, 8, 5]

Step 21 :
[1, 4, 2]
[3, 7, 0]
[6, 8, 5]

Step 22 :
[1, 4, 2]
[3, 7, 5]
[6, 8, 0]

Step 23 :
[1, 4, 2]
[3, 7, 5]
[6, 0, 8]

Step 24 :
[1, 4, 2]
[3, 0, 5]
[6, 7, 8]

Step 25 :
[1, 0, 2]
[3, 4

Step 1 :
[0, 6, 1]
[3, 4, 7]
[8, 5, 2]

Step 2 :
[3, 6, 1]
[0, 4, 7]
[8, 5, 2]

Step 3 :
[3, 6, 1]
[4, 0, 7]
[8, 5, 2]

Step 4 :
[3, 6, 1]
[4, 7, 0]
[8, 5, 2]

Step 5 :
[3, 6, 1]
[4, 7, 2]
[8, 5, 0]

Step 6 :
[3, 6, 1]
[4, 7, 2]
[8, 0, 5]

Step 7 :
[3, 6, 1]
[4, 7, 2]
[0, 8, 5]

Step 8 :
[3, 6, 1]
[0, 7, 2]
[4, 8, 5]

Step 9 :
[3, 6, 1]
[7, 0, 2]
[4, 8, 5]

Step 10 :
[3, 0, 1]
[7, 6, 2]
[4, 8, 5]

Step 11 :
[3, 1, 0]
[7, 6, 2]
[4, 8, 5]

Step 12 :
[3, 1, 2]
[7, 6, 0]
[4, 8, 5]

Step 13 :
[3, 1, 2]
[7, 6, 5]
[4, 8, 0]

Step 14 :
[3, 1, 2]
[7, 6, 5]
[4, 0, 8]

Step 15 :
[3, 1, 2]
[7, 6, 5]
[0, 4, 8]

Step 16 :
[3, 1, 2]
[0, 6, 5]
[7, 4, 8]

Step 17 :
[3, 1, 2]
[6, 0, 5]
[7, 4, 8]

Step 18 :
[3, 1, 2]
[6, 4, 5]
[7, 0, 8]

Step 19 :
[3, 1, 2]
[6, 4, 5]
[0, 7, 8]

Step 20 :
[3, 1, 2]
[0, 4, 5]
[6, 7, 8]

Step 21 :
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]

Step 1 :
[8, 7, 5]
[4, 1, 2]
[3, 0, 6]

Step 2 :
[8, 7, 5]
[4, 0, 2]
[3, 1, 6]

Step 3 :
[8, 7, 5]
[4, 2, 0]
[3, 1, 6]

Step 4 :
[8, 7, 0]
[4, 2, 5]
[3, 1, 6]

Step 5 :
[8, 0, 7]
[4, 2, 5]
[3, 1, 6]

Step 6 :
[8, 2, 7]
[4, 0, 5]
[3, 1, 6]

Step 7 :
[8, 2, 7]
[4, 1, 5]
[3, 0, 6]

Step 8 :
[8, 2, 7]
[4, 1, 5]
[3, 6, 0]

Step 9 :
[8, 2, 7]
[4, 1, 0]
[3, 6, 5]

Step 10 :
[8, 2, 0]
[4, 1, 7]
[3, 6, 5]

Step 11 :
[8, 0, 2]
[4, 1, 7]
[3, 6, 5]

Step 12 :
[8, 1, 2]
[4, 0, 7]
[3, 6, 5]

Step 13 :
[8, 1, 2]
[0, 4, 7]
[3, 6, 5]

Step 14 :
[0, 1, 2]
[8, 4, 7]
[3, 6, 5]

Step 15 :
[1, 0, 2]
[8, 4, 7]
[3, 6, 5]

Step 16 :
[1, 4, 2]
[8, 0, 7]
[3, 6, 5]

Step 17 :
[1, 4, 2]
[0, 8, 7]
[3, 6, 5]

Step 18 :
[1, 4, 2]
[3, 8, 7]
[0, 6, 5]

Step 19 :
[1, 4, 2]
[3, 8, 7]
[6, 0, 5]

Step 20 :
[1, 4, 2]
[3, 0, 7]
[6, 8, 5]

Step 21 :
[1, 4, 2]
[3, 7, 0]
[6, 8, 5]

Step 22 :
[1, 4, 2]
[3, 7, 5]
[6, 8, 0]

Step 23 :
[1, 4, 2]
[3, 7, 5]
[6, 0, 8]

Step 24 :
[1, 4, 2]
[3, 0, 5]
[6, 7, 8]

Step 25 :
[1, 0, 2]
[3, 4

Step 3 :
[3, 6, 1]
[4, 0, 7]
[8, 5, 2]

Step 4 :
[3, 6, 1]
[4, 7, 0]
[8, 5, 2]

Step 5 :
[3, 6, 1]
[4, 7, 2]
[8, 5, 0]

Step 6 :
[3, 6, 1]
[4, 7, 2]
[8, 0, 5]

Step 7 :
[3, 6, 1]
[4, 7, 2]
[0, 8, 5]

Step 8 :
[3, 6, 1]
[0, 7, 2]
[4, 8, 5]

Step 9 :
[3, 6, 1]
[7, 0, 2]
[4, 8, 5]

Step 10 :
[3, 0, 1]
[7, 6, 2]
[4, 8, 5]

Step 11 :
[3, 1, 0]
[7, 6, 2]
[4, 8, 5]

Step 12 :
[3, 1, 2]
[7, 6, 0]
[4, 8, 5]

Step 13 :
[3, 1, 2]
[7, 6, 5]
[4, 8, 0]

Step 14 :
[3, 1, 2]
[7, 6, 5]
[4, 0, 8]

Step 15 :
[3, 1, 2]
[7, 6, 5]
[0, 4, 8]

Step 16 :
[3, 1, 2]
[0, 6, 5]
[7, 4, 8]

Step 17 :
[3, 1, 2]
[6, 0, 5]
[7, 4, 8]

Step 18 :
[3, 1, 2]
[6, 4, 5]
[7, 0, 8]

Step 19 :
[3, 1, 2]
[6, 4, 5]
[0, 7, 8]

Step 20 :
[3, 1, 2]
[0, 4, 5]
[6, 7, 8]

Step 21 :
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

		Average_steps	Average_time
MisPlaced: 	29.0		2.67 seconds
Manhattan: 	29.0		10.32 seconds



# The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Solution of the first Scenario:
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### .
#### .
#### .
#### 0 1 2
#### 3 4 5
#### 6 7 8

                    Average_Steps    Average_Time      
 Miss Placed
 
 Manhattan Distance

